In [1]:

#초기 설정및 시스템 라이브러리
import platform
import warnings

# 데이터 시각화 라이브러리
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta
print(platform.system())
warnings.filterwarnings('ignore')

# 행,열,결과값 생략 없이 보기,세팅
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', None)

# setting Korean font
import platform
if platform.system() == 'Windows':
    plt.rcParams['font.family'] = 'Malgun Gothic'
elif platform.system() == 'Darwin':  # macOS
    plt.rcParams['font.family'] = 'AppleGothic'
else:  # Linux
    plt.rcParams['font.family'] = 'NanumGothic'

# statistic
from scipy import stats
from scipy.stats import shapiro, levene, ttest_ind, chi2_contingency, f_oneway
from scipy.stats import mannwhitneyu, fisher_exact, kruskal
from statsmodels.stats.multicomp import pairwise_tukeyhsd, MultiComparison
import pingouin as pg
import scikit_posthocs as sp
import statsmodels.api as sm
import statsmodels.formula.api as smf

from collections import Counter
from sklearn.datasets import load_diabetes
import scipy.stats as st
from pathlib import Path

# setting seed
np.random.seed(42)

Darwin


In [2]:
# Load Files

import pandas as pd

# 날짜 컬럼 방지
def read_csv_auto_dates(path):
    # 1) 전체 파일을 먼저 읽기
    temp = pd.read_csv(path, low_memory=False)
    
    # 2) _at 로 끝나는 컬럼 자동 탐지
    date_cols = [col for col in temp.columns if col.endswith('_at')]
    
    # 3) 다시 날짜 컬럼을 datetime 으로 읽기
    return pd.read_csv(path, parse_dates=date_cols, low_memory=False)

# Load Files
acq = read_csv_auto_dates("./cleaned_data/clean_acquisitions_final.csv")
deg = read_csv_auto_dates("./cleaned_data/clean_degrees_final.csv")
frs = read_csv_auto_dates("./cleaned_data/clean_fr_final.csv")
fds = read_csv_auto_dates("./cleaned_data/clean_funds_final.csv")
inv = read_csv_auto_dates("./cleaned_data/clean_investments_final.csv")
ipo = read_csv_auto_dates("./cleaned_data/clean_ipos_final.csv")
mil = read_csv_auto_dates("./cleaned_data/clean_milestones_final.csv")
obj= read_csv_auto_dates("./cleaned_data/clean_objects_final.csv")
off = read_csv_auto_dates("./cleaned_data/clean_offices_final.csv")
peo = read_csv_auto_dates("./cleaned_data/clean_people_final.csv")
rel = read_csv_auto_dates("./cleaned_data/clean_relationships_final.csv")
startup_profile = read_csv_auto_dates('./join_data/startup_profile.csv')
founder_profile = read_csv_auto_dates('./join_data/invested_founder_master.csv')
success_master = read_csv_auto_dates('./join_data/success_master.csv')


print("="*60)
print("dataset 로드 완료!")
print("="*60)

dataset 로드 완료!


### startup
- 총 스타트업 수(완료)
- 스타트업별 평균 투자 라운드 수(보류)
- 연도별 산업별 스타트업 분포
- 연도별 산업별 엑싯 유형별 분포(cat_obj_status)

In [3]:
###############################
# 변경자: 수아
# 변경일자: 25.12.24
# 변경
# 내용: 1. obj.founded_at 결측 보완 컬럼 생성(founded_at_estimated) 
##############################################################

import pandas as pd
import numpy as np

# ============================================================
# 스타트업 필터링 (투자만 한 회사 제거)
# ============================================================

# 0) obj에서 회사(c:)만
obj_c = obj[obj['objects_cfpr_id'].str.startswith('c:', na=False)].copy()

# 1) investments에서 investor로 등장한 c:
c_investors = set(
    inv.loc[
        inv['investor_cfp_id'].str.startswith('c:', na=False),
        'investor_cfp_id'
    ].dropna().unique()
)

# 2) investments에서 invested(투자받은 회사)로 등장한 c:
c_invested = set(
    inv.loc[
        inv['invested_c_id'].str.startswith('c:', na=False),
        'invested_c_id'
    ].dropna().unique()
)

# 3) investor에만 있는 c: (투자만 한 회사)
c_only_investor = c_investors - c_invested

# 4) obj의 c:에서 투자만 한 회사 제거
startup = obj_c[~obj_c['objects_cfpr_id'].isin(c_only_investor)].copy()

print(f"스타트업 수: {startup.shape[0]}")

import pandas as pd
import numpy as np

# ============================================================
# 0. 설립일 결측 스타트업 ID
# ============================================================

founded_na_ids = set(
    startup.loc[startup['founded_at'].isna(), 'objects_cfpr_id']
)

# ============================================================
# 1. milestone 기반 설립일 후보 (조건 분기 포함)
# ============================================================

FOUNDING_KEYWORDS = [
    'launch', 'release', 'founded', 'open', 'founder',
    'first', 'started', 'incorporated', 'established', 'foundation'
]
pattern = '|'.join(FOUNDING_KEYWORDS)

# 설립일 결측 회사의 milestone만
mil_sub = mil.loc[
    mil['mile_cfpr_id'].isin(founded_na_ids) &
    mil['milestone_at'].notna()
].copy()

# 회사별 첫 milestone
mil_first = (
    mil_sub
    .sort_values(by=['mile_cfpr_id', 'milestone_at'])
    .groupby('mile_cfpr_id', as_index=False)
    .first()
)

# startup의 first_milestone_at 조인
mil_first = mil_first.merge(
    startup[['objects_cfpr_id', 'first_milestone_at']],
    how='left',
    left_on='mile_cfpr_id',
    right_on='objects_cfpr_id'
)

# datetime 통일
mil_first['milestone_at'] = pd.to_datetime(mil_first['milestone_at'])
mil_first['first_milestone_at'] = pd.to_datetime(mil_first['first_milestone_at'])

# ------------------------------------------------------------
# CASE A: first_milestone_at 존재 → 날짜 일치 + 키워드
# ------------------------------------------------------------

case_a = mil_first.loc[
    mil_first['first_milestone_at'].notna() &
    (mil_first['milestone_at'] == mil_first['first_milestone_at']) &
    (mil_first['cat_mile_description'].str.contains(
        pattern, case=False, na=False, regex=True
    )),
    ['mile_cfpr_id', 'milestone_at']
]

# ------------------------------------------------------------
# CASE B: first_milestone_at 결측 → 첫 milestone + 키워드
# ------------------------------------------------------------

case_b = mil_first.loc[
    mil_first['first_milestone_at'].isna() &
    (mil_first['cat_mile_description'].str.contains(
        pattern, case=False, na=False, regex=True
    )),
    ['mile_cfpr_id', 'milestone_at']
]

# 병합
mil_founding = (
    pd.concat([case_a, case_b], ignore_index=True)
    .drop_duplicates(subset='mile_cfpr_id')
    .rename(columns={'milestone_at': 'milestone_founded_at'})
)

print(f"milestone 기반 설립일 보완 가능 회사 수: {mil_founding.shape[0]}")

# ============================================================
# 2. relationships 기반 설립일 후보 (Founder)
# ============================================================

rel_founder = rel.loc[
    (rel['cat_rel_title'] == 'Founder') &
    (rel['start_at'].notna())
].copy()

rel_founder_first = (
    rel_founder
    .sort_values(by=['rel_cf_id', 'start_at', 'sequence'])
    .groupby('rel_cf_id', as_index=False)
    .first()
    [['rel_cf_id', 'start_at']]
    .rename(columns={'start_at': 'founder_started_at'})
)

# ============================================================
# 3. startup 테이블에 후보 컬럼 조인
# ============================================================

startup = startup.merge(
    mil_founding,
    how='left',
    left_on='objects_cfpr_id',
    right_on='mile_cfpr_id'
).drop(columns=['mile_cfpr_id'], errors='ignore')

startup = startup.merge(
    rel_founder_first,
    how='left',
    left_on='objects_cfpr_id',
    right_on='rel_cf_id'
).drop(columns=['rel_cf_id'], errors='ignore')

# ============================================================
# 4. 최종 설립일 추정 컬럼 생성 (우선순위 적용)
# ============================================================

startup['founded_at_estimated'] = startup['founded_at']
startup['founded_at_source'] = None

# 1️⃣ objects.founded_at
mask = startup['founded_at'].notna()
startup.loc[mask, 'founded_at_source'] = 'objects'

# 2️⃣ milestone
mask = (
    startup['founded_at_estimated'].isna() &
    startup['milestone_founded_at'].notna()
)
startup.loc[mask, 'founded_at_estimated'] = startup.loc[mask, 'milestone_founded_at']
startup.loc[mask, 'founded_at_source'] = 'milestone_first_event'

# 3️⃣ relationships (Founder)
mask = (
    startup['founded_at_estimated'].isna() &
    startup['founder_started_at'].notna()
)
startup.loc[mask, 'founded_at_estimated'] = startup.loc[mask, 'founder_started_at']
startup.loc[mask, 'founded_at_source'] = 'founder_relationship'

# 4️⃣ first funding
mask = (
    startup['founded_at_estimated'].isna() &
    startup['first_funding_at'].notna()
)
startup.loc[mask, 'founded_at_estimated'] = startup.loc[mask, 'first_funding_at']
startup.loc[mask, 'founded_at_source'] = 'first_funding_at'

# ============================================================
# 5. 결과 요약
# ============================================================

print("\n설립일 출처 분포")
print(startup['founded_at_source'].value_counts(dropna=False))

still_missing = startup.loc[
    startup['founded_at_estimated'].isna(),
    'objects_cfpr_id'
]

print("--------------------------------------------------")
print(f"설립일 끝까지 추정 불가 스타트업 수: {still_missing.nunique()}")
print("샘플 ID:", still_missing.unique()[:10])
print("--------------------------------------------------")


print(f"설립일 결측 id 개수:{startup['founded_at'].isna().sum()}")
print(f"설립일 결측 보완 불가 id 개수: {startup['founded_at_estimated'].isna().sum()}")


스타트업 수: 194151
milestone 기반 설립일 보완 가능 회사 수: 143

설립일 출처 분포
founded_at_source
objects                  90508
None                     88689
founder_relationship      8060
first_funding_at          6751
milestone_first_event      143
Name: count, dtype: int64
--------------------------------------------------
설립일 끝까지 추정 불가 스타트업 수: 88689
샘플 ID: ['c:100' 'c:100042' 'c:10005' 'c:10016' 'c:100211' 'c:100231' 'c:100265'
 'c:1004' 'c:10050' 'c:100589']
--------------------------------------------------
설립일 결측 id 개수:103643
설립일 결측 보완 불가 id 개수: 88689


In [4]:
startup[['objects_cfpr_id', 'status', 'founded_at', 'founded_at_estimated', 'founded_at_source']]

,objects_cfpr_id,status,founded_at,founded_at_estimated,founded_at_source
0,c:1,operating,2005-10-17,2005-10-17,objects
1,c:10,acquired,NaT,2006-05-01,founder_relationship
2,c:100,acquired,NaT,NaT,None
3,c:10000,operating,2008-07-26,2008-07-26,objects
4,c:10001,operating,2008-07-26,2008-07-26,objects
...,...,...,...,...,...
194146,c:99940,operating,2007-01-01,2007-01-01,objects
194147,c:9995,operating,2007-11-01,2007-11-01,objects
194148,c:9996,operating,1959-01-01,1959-01-01,objects
194149,c:9997,operating,2008-07-01,2008-07-01,objects


In [5]:
startup[(startup['founded_at'].isna()) & (startup['founded_at_estimated'].isna())]['cat_obj_status'].value_counts()

cat_obj_status
operating    84496
acquired      3853
ipo            248
closed          92
Name: count, dtype: int64

In [6]:
# 불필요한 컬럼 정리
startup = startup[[
    'objects_cfpr_id',
    'founded_at',
    'closed_at',
    'description',
    'tag_list',
    'country_code',
    'first_investment_at',
    'last_investment_at',
    'first_funding_at',
    'last_funding_at',
    'funding_rounds',
    'funding_total_usd',
    'first_milestone_at',
    'last_milestone_at',
    'milestones',
    'relationships',
    'cat_obj_status',
    'cat_obj_overview',
    'obj_state_filled',
    'obj_city_fixed',
    'obj_category_filled',
    'is_obj_funding_total_usd_private',
    'is_obj_funding_rounds_private',
    'founded_at_estimated',
    'founded_at_source'
]]


In [7]:
# csv 저장
startup.to_csv("./cleaned_data/startup.csv", index=False, encoding="utf-8-sig")

# 태블로를 위한 테이블 조인

In [8]:
frs.groupby('fr_c_id')['funding_round_id'].count().mean()

np.float64(1.657158959266101)

In [9]:
startup.groupby('objects_cfpr_id')['funding_rounds'].mean().mean()

np.float64(0.2639259032723142)

In [10]:

# Load Files

import pandas as pd

# 날짜 컬럼 방지
def read_csv_auto_dates(path):
    # 1) 전체 파일을 먼저 읽기
    temp = pd.read_csv(path, low_memory=False)
    
    # 2) _at 로 끝나는 컬럼 자동 탐지
    date_cols = [col for col in temp.columns if col.endswith('_at')]
    
    # 3) 다시 날짜 컬럼을 datetime 으로 읽기
    return pd.read_csv(path, parse_dates=date_cols, low_memory=False)

# Load Files
stu = read_csv_auto_dates("./cleaned_data/startup.csv")
startup_profile = read_csv_auto_dates("./join_data/startup_profile.csv")

print("="*60)
print("dataset 로드 완료!")
print("="*60)

dataset 로드 완료!


In [11]:
#############################################################
# 변경자: 수아
# 변경일자: 25.12.24
# 변경
# 내용: 1. 태블로를 위한 테이블 조인
#       1) funding_rounds←startup(objects): grain(funding_round_id x startup_id)
#       2) startup_profile←success_master: grain(startup_id)
#       3) objects←offices: grain(startup_id)
#       4) ipos←objects: grain(ipo_id x startup_id)
#       5) acquisitions←objects: grain(acquisition_id x startup_id)
#       6) funding_round←success_master: grain(funding_round_id x startup_id) -> 라운드별 투자 성공률: 스타트업의 라운드별 성장, 엑싯 건수 / 해당 라운드 전체 투자 건수
#       7) funding_rounds←objects←ipos←acquisitions: grain(startup_id) -> 스타트업의 첫 투자 이벤트와 첫 엑싯 이벤트만 남기기
##############################################################

### funding_rounds ← startup
- 연도별 산업별 투자 건수 
- 연도별 산업별 투자 라운드별 스타트업 분포
- 산업별 라운드 성장 건수

In [12]:
# =======================================
# 1) funding_rounds←startup(objects) 조인
# grain(funding_round_id x startup_id)
# =======================================

# 필요한 컬럼 추출
base_frs = frs[['funding_round_id', 'fr_c_id', 'funded_at',
       'raised_amount_usd', 'participants', 'is_first_round',
       'is_last_round', 'funded_year', 'cat_fr_type',
       'num_fr_type', 'is_fr_raised_private']]

# merge
frs_stu = (base_frs
           .merge(stu, left_on='fr_c_id', right_on = 'objects_cfpr_id', how='left'))

# 행 수 확인
print(f"조인 전 행 수: {frs.shape}")
print(f"조인 후 행 수: {frs_stu.shape}")

# csv 저장
frs_stu.to_csv("./join_data/frs_obj(startup_filtered).csv")
print("조인 파일 저장 완료!")

조인 전 행 수: (52928, 17)
조인 후 행 수: (52928, 36)
조인 파일 저장 완료!


### startup_profile ← success_master
- 전체 투자 성공률
- 산업별 투자 성공률
- 연도별 산업별 스타트업 기준 투자 성공률

In [13]:
# =======================================
# 2) startup_profile ← success_master
# grain: startup_id (objects_cfpr_id)
# funding_round_id는 notna 여부로 요약
# =======================================

print(f"조인 전 행 수: {startup_profile.shape}")

# ------------------------------------------------
# 1. 베이스 생성 (startup grain)
# ------------------------------------------------
base_stu_profile = startup_profile[[
    'objects_cfpr_id',
    'founded_at',
    'closed_at',
    'description',
    'country_code',
    'obj_city_fixed',
    'first_funding_at',
    'last_funding_at',
    'funding_rounds',
    'funding_total_usd',
    'cat_obj_status',
    'obj_category_filled',
    'cat_obj_overview',
    'obj_state_filled',
    'is_obj_funding_total_usd_private',
    'rel_cf_id',
    'relationship_growth'
]]

# ------------------------------------------------
# 2. success_master 집계 (startup grain)
#    - success_flag: 스타트업 기준
#    - has_funding_round: funding_round_id notna 여부
# ------------------------------------------------
sm_agg = (
    success_master
        .groupby('objects_cfpr_id')
        .agg(
            success_flag=('success_flag', 'max'),
            has_funding_round=('funding_round_id', lambda x: x.notna().any())
        )
        .reset_index()
)

# ------------------------------------------------
# 3. 설립일 추정 컬럼
# ------------------------------------------------
stu_founded = stu[[
    'objects_cfpr_id',
    'founded_at_estimated',
    'founded_at_source'
]]

# ------------------------------------------------
# 4. merge
# ------------------------------------------------
stu_prof_succ = (
    base_stu_profile
        .merge(sm_agg, on='objects_cfpr_id', how='left')
        .merge(stu_founded, on='objects_cfpr_id', how='left')
)

# ------------------------------------------------
# 5. 결과 확인
# ------------------------------------------------
print("startup_profile ← success_master 집계 조인 완료")
print(f"조인 후 행 수: {stu_prof_succ.shape}")

# ------------------------------------------------
# 6. CSV 저장
# ------------------------------------------------
stu_prof_succ.to_csv(
    "./join_data/startup_profile_sm.csv",
    index=False,
    encoding="utf-8-sig"
)

print("csv 저장 완료")


조인 전 행 수: (194151, 25)
startup_profile ← success_master 집계 조인 완료
조인 후 행 수: (194151, 21)
csv 저장 완료


In [14]:
stu_prof_succ[stu_prof_succ['has_funding_round']==False]['objects_cfpr_id'].nunique()

162518

In [15]:
stu_prof_succ[stu_prof_succ['has_funding_round']==True]['objects_cfpr_id'].nunique()

31633

### startup ← offices: lattitude & longitude

In [16]:
# =======================================
# 3) objects(startup_filetered)←offices
# grain(startup_id)
# =======================================

print(f"조인 전 행 수: {stu.shape}")

# 베이스 생성
base_stu_off = stu[['objects_cfpr_id', 'founded_at', 'closed_at', 'description', 'tag_list',
       'country_code', 'relationships', 'cat_obj_status', 'cat_obj_overview',
       'obj_state_filled', 'obj_city_fixed', 'obj_category_filled',
       'founded_at_estimated', 'founded_at_source']]


# off 연결 위한 컬럼 추가
off_con_1 = startup_profile[['offices_c_id', 'office_id', 'office_city']]
off_con_2 = off[['office_id', 'country_code', 'latitude', 'longitude', 'offices_state_filled']]

# 위도/경도 컬럼 추가
off_con_3 = off_con_1.merge(off_con_2, on='office_id', how='left').dropna()


# merge
stu_off = (base_stu_off
                 .merge(off_con_3, left_on='objects_cfpr_id', right_on='offices_c_id', how='left')
                 )

print("objects(startup_filetered)←offices 조인 완료")
print(f"objects(startup_filetered)←offices 조인 후 행 수:{stu_off.shape}")

# csv 저장
stu_off.to_csv("./join_data/objects(startup_filtered)_offices.csv")
print("csv 저장 완료")

조인 전 행 수: (194151, 25)
objects(startup_filetered)←offices 조인 완료
objects(startup_filetered)←offices 조인 후 행 수:(194151, 21)
csv 저장 완료


### success_master: time_to_exit

In [17]:
# ============================================================
# 연도별 · 산업별 time_to_exit 분석용 테이블 생성
# source: success_master (단독 사용)
# grain: objects_cfpr_id (startup)
# exit 정의:
#   - IPO가 먼저면: first_public_at - first_funding_at
#   - M&A가 먼저면: acquired_at - first_funding_at
# ============================================================

# ============================================================
# 0. 스타트업 필터링 (투자만 한 회사 제거)
# ============================================================

# obj에서 회사(c:)만
obj_c = obj[obj['objects_cfpr_id'].str.startswith('c:', na=False)].copy()

# investments에서 investor로 등장한 c:
c_investors = set(
    inv.loc[
        inv['investor_cfp_id'].str.startswith('c:', na=False),
        'investor_cfp_id'
    ].dropna().unique()
)

# investments에서 invested(투자받은 회사)로 등장한 c:
c_invested = set(
    inv.loc[
        inv['invested_c_id'].str.startswith('c:', na=False),
        'invested_c_id'
    ].dropna().unique()
)

# investor에만 있는 c: (투자만 한 회사)
c_only_investor = c_investors - c_invested

# 실제 스타트업만 남김
startup_filtered = obj_c[
    ~obj_c['objects_cfpr_id'].isin(c_only_investor)
].copy()

print(f"투자만 한 회사 제거 후 스타트업 수: {startup_filtered.shape[0]}")

# ============================================================
# 1. success_master에 스타트업 필터 적용
# ============================================================

sm_startup = success_master[
    success_master['objects_cfpr_id'].isin(
        startup_filtered['objects_cfpr_id']
    )
].copy()

print(f"success_master 필터 후 행 수: {sm_startup.shape}")

# ============================================================
# 2. 회사(grain) 기준 exit 분석 베이스 생성
# ============================================================

startup_exit_base = (
    sm_startup[[
        'objects_cfpr_id',
        'obj_category_filled',
        'first_funding_at',
        'first_public_at',
        'acquired_at'
    ]]
    .drop_duplicates(subset=['objects_cfpr_id'])
    .copy()
)

# ============================================================
# 3. 날짜 타입 정리
# ============================================================

for col in ['first_funding_at', 'first_public_at', 'acquired_at']:
    startup_exit_base[col] = pd.to_datetime(
        startup_exit_base[col],
        errors='coerce'
    )

# ============================================================
# 4. 회사별 최초 exit 이벤트 결정 (IPO vs M&A)
# ============================================================

startup_exit_base['exit_date'] = np.where(
    startup_exit_base['first_public_at'].notna() &
    (
        startup_exit_base['acquired_at'].isna() |
        (startup_exit_base['first_public_at'] <= startup_exit_base['acquired_at'])
    ),
    startup_exit_base['first_public_at'],   # IPO가 먼저
    startup_exit_base['acquired_at']         # M&A가 먼저
)

# ============================================================
# 5. exit_date 또는 first_funding_at 결측 제거
#    → 회수기간 정의 불가능 케이스 제외
# ============================================================

startup_exit_base = startup_exit_base[
    startup_exit_base['exit_date'].notna() &
    startup_exit_base['first_funding_at'].notna()
].copy()

# ============================================================
# 6. time_to_exit 계산 (개월 단위)
# ============================================================

startup_exit_base['time_to_exit_months'] = (
    (startup_exit_base['exit_date'] - startup_exit_base['first_funding_at'])
    .dt.days / 30
)

# ============================================================
# 7. 비정상 회수기간 제거 (exit < first_funding)
# ============================================================

startup_exit_base = startup_exit_base[
    startup_exit_base['time_to_exit_months'] >= 0
].copy()

# ============================================================
# 8. exit 연도 파생
# ============================================================

startup_exit_base['exit_year'] = (
    startup_exit_base['exit_date'].dt.year
)

# ============================================================
# 9. 최종 분석 테이블 (연도별 · 산업별 분포용)
# ============================================================

time_to_exit_final = startup_exit_base[[
    'objects_cfpr_id',
    'obj_category_filled',
    'exit_year',
    'time_to_exit_months'
]]

print("연도별 · 산업별 time_to_exit 분석 테이블 생성 완료")
print(time_to_exit_final.shape)

# ============================================================
# (선택) 요약 통계 테이블
# ============================================================

exit_summary = (
    time_to_exit_final
        .groupby(['exit_year', 'obj_category_filled'])
        .agg(
            n_startups=('objects_cfpr_id', 'nunique'),
            median_time_to_exit=('time_to_exit_months', 'median'),
            mean_time_to_exit=('time_to_exit_months', 'mean')
        )
        .reset_index()
)

print("연도별 · 산업별 time_to_exit 요약 테이블 생성 완료")
print(exit_summary.shape)

# ============================================================
# csv 파일 저장
# ============================================================

time_to_exit_final.to_csv(
    "./cleaned_data/time_to_exit.csv",
    index=False
)


투자만 한 회사 제거 후 스타트업 수: 194151
success_master 필터 후 행 수: (215055, 58)
연도별 · 산업별 time_to_exit 분석 테이블 생성 완료
(515, 4)
연도별 · 산업별 time_to_exit 요약 테이블 생성 완료
(138, 5)


### ipo ← success_master: 산업별 상장 건수

In [18]:
# ============================================================
# IPO 이벤트 기준 분석용 테이블 생성
# grain: ipo_id (IPO 이벤트 단위)
# 목적:
#  - 산업별 IPO 이벤트 수
#  - 연도별 IPO 트렌드
# ============================================================

# 1. 컬럼 정리
ipo_cols = ipo[['ipo_id', 'ipos_c_id', 'public_at', 'is_ipos_public_at_missing']]

stu_cols = stu[['objects_cfpr_id', 'founded_at', 'closed_at', 'country_code',
       'first_funding_at', 'last_funding_at', 'funding_rounds', 'first_milestone_at', 'last_milestone_at',
       'milestones', 'relationships', 'cat_obj_status', 'cat_obj_overview',
       'obj_state_filled', 'obj_city_fixed', 'obj_category_filled','is_obj_funding_rounds_private',
       'founded_at_estimated', 'founded_at_source']]

# 3. IPO 이벤트 기준으로 startup 정보 조인
ipo_stu = (
    ipo_cols
        .merge(
            stu_cols,
            how='left',
            left_on='ipos_c_id',
            right_on='objects_cfpr_id'
        )
)

print("IPO 이벤트 기준 + 산업/투자 히스토리 조인 완료")
print(ipo_stu.shape)

# ============================================================
# 4. IPO 연도 파생 (Tableau 필터용)
# ============================================================

ipo_stu['ipo_year'] = pd.to_datetime(
    ipo_stu['public_at'],
    errors='coerce'
).dt.year

# ============================================================
# (선택) 분석용 컬럼 정리된 뷰
#  - grain 유지
#  - Tableau에 바로 연결 가능
# ============================================================

ipo_event_final = ipo_stu[[
    # IPO 이벤트 키
    'ipo_id',
    'ipos_c_id',

    # IPO 이벤트 정보
    'public_at',
    'ipo_year',

    # 회사/산업 정보
    'obj_category_filled',
    'cat_obj_status',
    'founded_at',

    # 투자 히스토리 (라운드/규모)
    'first_funding_at',
    'last_funding_at',
    'funding_rounds',
    'is_obj_funding_rounds_private'
]]

print("IPO 이벤트 분석 최종 테이블 생성 완료")
print(ipo_event_final.shape)


# csv 저장
ipo_stu.to_csv("./join_data/ipo_startup.csv")
print("조인 파일 csv 저장 완료")

IPO 이벤트 기준 + 산업/투자 히스토리 조인 완료
(1259, 23)
IPO 이벤트 분석 최종 테이블 생성 완료
(1259, 11)
조인 파일 csv 저장 완료


### acquisition ← startup: 산업별 인수합병 건수

In [19]:
acq.columns

Index(['acquisition_id', 'acquiring_c_id', 'acquired_c_id', 'term_code',
       'price_amount', 'price_currency_code', 'acquired_at',
       'is_acq_price_private', 'is_acquisitions_acq_at_missing',
       'price_amount_usd', 'acqusition_currency_rate'],
      dtype='object')

In [20]:
# ============================================================
# M&A 이벤트 기준 분석용 테이블 생성
# grain: acquisition_id (M&A 이벤트 단위)
# 목적:
#  - 산업별 M&A 이벤트 수
#  - 연도별 M&A 트렌드
# ============================================================

# 1. 컬럼 정리
acq_cols = acq[['acquisition_id', 'acquiring_c_id', 'acquired_c_id', 'acquired_at',
                'is_acquisitions_acq_at_missing']]

stu_cols = stu[['objects_cfpr_id', 'founded_at', 'closed_at', 'country_code',
       'first_funding_at', 'last_funding_at', 'funding_rounds', 'first_milestone_at', 'last_milestone_at',
       'milestones', 'relationships', 'cat_obj_status', 'cat_obj_overview',
       'obj_state_filled', 'obj_city_fixed', 'obj_category_filled','is_obj_funding_rounds_private',
       'founded_at_estimated', 'founded_at_source']]

# 3. M&A 이벤트 기준으로 startup 정보 조인
acq_stu = (
    acq_cols
        .merge(
            stu_cols,
            how='left',
            left_on='acquired_c_id',
            right_on='objects_cfpr_id'
        )
)

print("M&A 이벤트 기준 + 산업/투자 히스토리 조인 완료")
print(acq_stu.shape)

# ============================================================
# 4. M&A 연도 파생 (Tableau 필터용)
# ============================================================

acq_stu['acq_year'] = pd.to_datetime(
    acq_stu['acquired_at'],
    errors='coerce'
).dt.year


# csv 저장
acq_stu.to_csv("./join_data/acquisition_startup.csv")
print("조인 파일 csv 저장 완료")

M&A 이벤트 기준 + 산업/투자 히스토리 조인 완료
(9562, 24)
조인 파일 csv 저장 완료


### 투자라운드 성장 건수: success_master

In [23]:
# ============================================================
# 산업별 투자 라운드 성장 건수 집계 (최종 확정본)
# 규칙:
# - 성장 라운드: seed, series-a, series-b, series-c+
# - 같은 회사 × 같은 stage 중복 → 1번만 인정
# - 성장 건수 = 고유 stage 개수 - 1
# - Exit 있는 회사: exit 이전 stage만 사용
# - Exit 없는 회사: 전체 stage 사용
# ============================================================

import pandas as pd

# ------------------------------------------------------------
# STEP 0. 스타트업 필터링 (투자만 한 회사 제거)
# ------------------------------------------------------------

obj_c = obj[obj['objects_cfpr_id'].str.startswith('c:', na=False)].copy()

c_investors = set(
    inv.loc[
        inv['investor_cfp_id'].str.startswith('c:', na=False),
        'investor_cfp_id'
    ].dropna().unique()
)

c_invested = set(
    inv.loc[
        inv['invested_c_id'].str.startswith('c:', na=False),
        'invested_c_id'
    ].dropna().unique()
)

c_only_investor = c_investors - c_invested

startup = obj_c[
    ~obj_c['objects_cfpr_id'].isin(c_only_investor)
].copy()

# ------------------------------------------------------------
# STEP 1. success_master 준비
# ------------------------------------------------------------

sm = success_master.copy()

date_cols = ['funded_at', 'acquired_at', 'first_public_at']
for col in date_cols:
    sm[col] = pd.to_datetime(sm[col], errors='coerce')

sm = sm[sm['objects_cfpr_id'].isin(startup['objects_cfpr_id'])].copy()

# 성장 라운드만 사용
growth_rounds = ['seed', 'series-a', 'series-b', 'series-c+']
sm = sm[sm['cat_fr_type'].isin(growth_rounds)].copy()

# ------------------------------------------------------------
# STEP 2. 회사별 exit_date 계산
# ------------------------------------------------------------

exit_df = (
    sm[['objects_cfpr_id', 'acquired_at', 'first_public_at']]
    .drop_duplicates()
)

exit_df['exit_date'] = exit_df[['acquired_at', 'first_public_at']].min(axis=1)

# exit_date 병합
sm = sm.merge(
    exit_df[['objects_cfpr_id', 'exit_date']],
    on='objects_cfpr_id',
    how='left'
)

# ------------------------------------------------------------
# STEP 3. 성장 건수 계산 함수
# ------------------------------------------------------------

stage_order = {
    'seed': 0,
    'series-a': 1,
    'series-b': 2,
    'series-c+': 3
}

def calc_growth_count(df):
    exit_date = df['exit_date'].iloc[0]

    # Exit 있는 회사 → exit 이전만 사용
    if pd.notna(exit_date):
        df = df.dropna(subset=['funded_at'])
        df = df[df['funded_at'] <= exit_date]

    # stage를 순서 값으로 변환
    stages = (
        df['cat_fr_type']
        .map(stage_order)
        .dropna()
        .unique()
    )

    # 성장 건수 = 고유 stage 수 - 1
    return max(len(stages) - 1, 0)

# ------------------------------------------------------------
# STEP 4. 회사별 성장 건수 계산
# ------------------------------------------------------------

company_growth = (
    sm.groupby('objects_cfpr_id')
      .apply(calc_growth_count)
      .reset_index(name='growth_event_count')
)

# 산업 정보 병합
company_growth = company_growth.merge(
    startup[['objects_cfpr_id', 'obj_category_filled']],
    on='objects_cfpr_id',
    how='left'
)

# ------------------------------------------------------------
# STEP 5. 산업별 성장 건수 집계 (최종 결과)
# ------------------------------------------------------------

industry_growth_counts = (
    company_growth
        .groupby('obj_category_filled')
        .agg(
            total_growth_events=('growth_event_count', 'sum'),
            n_companies=('objects_cfpr_id', 'nunique'),
            avg_growth_events=('growth_event_count', 'mean')
        )
        .reset_index()
        .sort_values('total_growth_events', ascending=False)
)


industry_growth_counts.to_csv("./cleaned_data/funding_round_growth_1.csv")
industry_growth_counts


,obj_category_filled,total_growth_events,n_companies,avg_growth_events
40,software,715,3890,0.183805
10,enterprise,462,1245,0.371084
44,web,400,2481,0.161225
0,advertising,376,981,0.383282
4,biotech,365,1594,0.228984
26,mobile,357,1682,0.212247
8,ecommerce,260,1296,0.200617
14,games_video,211,1051,0.200761
2,analytics,183,561,0.326203
5,cleantech,169,578,0.292388


### 연도/산업/투자 라운드 기준 투자 성공률

In [63]:
import pandas as pd
import numpy as np

# =================================
# 0. 실제 스타트업만 필터링
# =================================
obj_c = obj[obj['objects_cfpr_id'].str.startswith('c:', na=False)]

c_investors = set(inv.loc[
    inv['investor_cfp_id'].str.startswith('c:', na=False),
    'investor_cfp_id'
])

c_invested = set(inv.loc[
    inv['invested_c_id'].str.startswith('c:', na=False),
    'invested_c_id'
])

startup_ids = set(
    obj_c[~obj_c['objects_cfpr_id'].isin(c_investors - c_invested)]
    ['objects_cfpr_id']
)

# =================================
# 1. 투자 라운드 이벤트 모집단
# =================================
fr = success_master[
    success_master['objects_cfpr_id'].isin(startup_ids) &
    success_master['funding_round_id'].notna() &
    success_master['funded_at'].notna()
].copy()

fr['funded_at'] = pd.to_datetime(fr['funded_at'], errors='coerce')
fr['round_year'] = fr['funded_at'].dt.year

# =================================
# 2. 라운드 최초 진입만 남기기
# =================================
fr = fr.sort_values(['objects_cfpr_id', 'num_fr_type', 'funded_at'])

fr_entry = fr.drop_duplicates(
    subset=['objects_cfpr_id', 'cat_fr_type'],
    keep='first'
)

# =================================
# 3. 회사별 첫 엑싯 시점
# =================================
exit_dates = success_master[['objects_cfpr_id', 'acquired_at', 'first_public_at']].copy()
exit_dates['acquired_at'] = pd.to_datetime(exit_dates['acquired_at'], errors='coerce')
exit_dates['first_public_at'] = pd.to_datetime(exit_dates['first_public_at'], errors='coerce')

exit_dates['first_exit_date'] = exit_dates[['acquired_at', 'first_public_at']].min(axis=1)

fr_entry = fr_entry.merge(
    exit_dates[['objects_cfpr_id', 'first_exit_date']],
    on='objects_cfpr_id',
    how='left'
)

# 엑싯 이후 라운드 제거
fr_entry = fr_entry[
    fr_entry['first_exit_date'].isna() |
    (fr_entry['funded_at'] <= fr_entry['first_exit_date'])
]

# =================================
# 4. 성공 여부 판단 (라운드 기준)
# =================================
max_round = fr.groupby('objects_cfpr_id')['num_fr_type'].max()

fr_entry['max_round'] = fr_entry['objects_cfpr_id'].map(max_round)

fr_entry['is_success'] = (
    (fr_entry['max_round'] > fr_entry['num_fr_type']) |
    (
        fr_entry['first_exit_date'].notna() &
        (fr_entry['funded_at'] <= fr_entry['first_exit_date'])
    )
).astype(int)

# =================================
# 5. Table A 완성
# =================================
table_A = fr_entry[[
    'objects_cfpr_id',
    'obj_category_filled',
    'cat_fr_type',
    'round_year',
    'is_success'
]]

table_A.to_csv("./cleaned_data/table_A.csv", index=False)


In [64]:
table_A

,objects_cfpr_id,obj_category_filled,cat_fr_type,round_year,is_success
0,c:1,web,series-a,2005,0
1,c:1,web,series-a,2005,0
2,c:1,web,series-a,2005,0
3,c:10015,health,series-a,2008,0
4,c:10015,health,series-a,2008,0
...,...,...,...,...,...
27951,c:9939,web,seed,2008,0
27952,c:9949,games_video,series-c+,2009,1
27953,c:9949,games_video,series-c+,2009,1
27954,c:9949,games_video,series-c+,2009,1


### 연도/산업/스타트업 기준 투자 성공률

In [67]:
import pandas as pd
import numpy as np

# ==============================
# 0. 실제 스타트업 ID 정의
# ==============================
obj_c = obj[obj['objects_cfpr_id'].str.startswith('c:', na=False)]

c_investors = set(
    inv.loc[inv['investor_cfp_id'].str.startswith('c:', na=False), 'investor_cfp_id']
)
c_invested = set(
    inv.loc[inv['invested_c_id'].str.startswith('c:', na=False), 'invested_c_id']
)

c_only_investor = c_investors - c_invested

startup_ids = set(
    obj_c[~obj_c['objects_cfpr_id'].isin(c_only_investor)]['objects_cfpr_id']
)

# ==============================
# 1. 투자 이벤트 (모집단)
# ==============================
base = success_master[
    success_master['objects_cfpr_id'].isin(startup_ids) &
    success_master['funding_round_id'].notna() &
    success_master['funded_at'].notna()
].copy()

base['funded_at'] = pd.to_datetime(base['funded_at'], errors='coerce')
base['funded_year'] = base['funded_at'].dt.year

# 스타트업 × 투자연도 단위
base = base[[
    'objects_cfpr_id',
    'obj_category_filled',
    'funded_year'
]].drop_duplicates()

# ==============================
# 2. 성공 이벤트 생성
# ==============================

events = success_master[
    success_master['objects_cfpr_id'].isin(startup_ids)
][[
    'objects_cfpr_id',
    'funded_at',
    'num_fr_type',
    'acquired_at',
    'first_public_at'
]].copy()

events['funded_at'] = pd.to_datetime(events['funded_at'], errors='coerce')
events['acquired_at'] = pd.to_datetime(events['acquired_at'], errors='coerce')
events['first_public_at'] = pd.to_datetime(events['first_public_at'], errors='coerce')

# (1) 다음 라운드 진입 성공
events = events.sort_values(['objects_cfpr_id', 'num_fr_type', 'funded_at'])
events['prev_round'] = events.groupby('objects_cfpr_id')['num_fr_type'].shift(1)

round_success = events[
    (events['num_fr_type'] > events['prev_round']) &
    events['funded_at'].notna()
][['objects_cfpr_id', 'funded_at']]

round_success['success_date'] = round_success['funded_at']

# (2) IPO / M&A 성공
exit_success = events.copy()
exit_success['exit_date'] = exit_success[['acquired_at', 'first_public_at']].min(axis=1)

exit_success = exit_success.dropna(subset=['exit_date'])[
    ['objects_cfpr_id', 'exit_date']
].rename(columns={'exit_date': 'success_date'})

# (3) 성공 이벤트 통합
success_events = pd.concat(
    [round_success[['objects_cfpr_id', 'success_date']], exit_success],
    ignore_index=True
)

# ==============================
# 3. 투자 시점 이후 성공 여부 판정
# ==============================
table_B = base.merge(
    success_events,
    on='objects_cfpr_id',
    how='left'
)

# 투자 연도 이후(같은 해 포함)에 성공했는가?
table_B['is_success'] = (
    table_B['success_date'].dt.year >= table_B['funded_year']
).fillna(False).astype(int)

# ==============================
# 4. 최종 Table B
# ==============================
table_B = table_B[[
    'objects_cfpr_id',
    'obj_category_filled',
    'funded_year',
    'is_success'
]].drop_duplicates()

table_B.to_csv("./cleaned_data/table_B.csv", index=False)


In [68]:
table_B

,objects_cfpr_id,obj_category_filled,funded_year,is_success
0,c:1,web,2005,0
1,c:10015,health,2008,0
2,c:100189,mobile,2003,0
3,c:10026,games_video,2007,0
4,c:1003,web,2007,0
...,...,...,...,...
15209,c:9905,software,2005,1
15211,c:9920,web,2008,0
15212,c:9937,web,2006,0
15213,c:9939,web,2008,0
